In [ ]:
#Environment Requirements

In [2]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2
datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2021-12-30 07:58:26--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  22.2MB/s    in 2.8s    

2021-12-30 07:58:29 (22.2 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import pandas as pd

In [45]:
df=pd.DataFrame(columns=['frame', 'EAR', 'Blink_Counter'])

In [46]:
df

,frame,EAR,Blink_Counter


In [ ]:
#Backend people can ignore this part of the code.
# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from google.colab.patches import cv2_imshow
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
 
def eye_aspect_ratio(eye):

	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
 
	# return the eye aspect ratio
	return ear
 
p = '/content/shape_predictor_68_face_landmarks.dat'
v = "/content/drive/MyDrive/Video_AU_RML/Converted_MP4/AIC_301.mp4"

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3
 
# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector () 
predictor = dlib.shape_predictor (p) 
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
print("[INFO] starting video stream thread...")

vs = FileVideoStream (v) .start () 
fileStream = True
time.sleep(1.0)
frame_num=0
while True:
	# if this is a file video stream, then we need to check if
	# there any more frames left in the buffer to process
	if fileStream and not vs.more():
		break

	# grab the frame from the threaded video file stream, resize
	# it, and convert it to grayscale
	# channels)
	frame = vs.read()
	frame_num+=1
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# detect faces in the grayscale frame
	rects = detector(gray, 0)

	# loop over the face detections
	for rect in rects:
		# determine the facial landmarks for the face region, then
		# convert the facial landmark (x, y)-coordinates to a NumPy
		# array
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)

		# extract the left and right eye coordinates, then use the
		# coordinates to compute the eye aspect ratio for both eyes
		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]
		leftEAR = eye_aspect_ratio(leftEye)
		rightEAR = eye_aspect_ratio(rightEye)

		# average the eye aspect ratio together for both eyes
		ear = (leftEAR + rightEAR) / 2.0

		# compute the convex hull for the left and right eye, then
		# visualize each of the eyes
		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		cv2.drawContours(gray, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(gray, [rightEyeHull], -1, (0, 255, 0), 1)

		# check to see if the eye aspect ratio is below the blink
		# threshold, and if so, increment the blink frame counter
		if ear < EYE_AR_THRESH:
			COUNTER += 1

		# otherwise, the eye aspect ratio is not below the blink
		# threshold
		else:
			# if the eyes were closed for a sufficient number of
			# then increment the total number of blinks
			if COUNTER >= EYE_AR_CONSEC_FRAMES:
				TOTAL += 1

			# reset the eye frame counter
			COUNTER = 0

		# draw the total number of blinks on the frame along with
		# the computed eye aspect ratio for the frame
		cv2.putText(gray, "Blinks: {}".format(TOTAL), (10, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		cv2.putText(gray, "EAR: {:.2f}".format(ear), (300, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
	df=df.append({'frame':frame_num, 'EAR': ear, 'Blink_Counter':TOTAL } ,ignore_index=True)
	# show the frame
	cv2_imshow(gray)
	key = cv2.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#df=df.set_index('frame')
df.plot(x='frame', y='EAR')

In [ ]:
df.plot(x='frame', y='Blink_Counter')

In [102]:
##Python Script for backend integration
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
#from google.colab.patches import cv2_imshow

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from imutils import face_utils
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2

def eye_aspect_ratio(eye):

	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
 
	# return the eye aspect ratio
	return ear

def blink_detector(p,v):
  #Parameter p is the location of the facial landmarks dat file
  #Parameter q is the location of the video file that we want to analyse.
  EYE_AR_THRESH = 0.
  EYE_AR_CONSEC_FRAMES = 3
 
  # initialize the frame counters and the total number of blinks
  COUNTER = 0
  TOTAL = 0
  #print("[INFO] loading facial landmark predictor...")
  detector = dlib.get_frontal_face_detector () 
  predictor = dlib.shape_predictor (p) 
  (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
  (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
  #print("[INFO] starting video stream thread...")
  vs = FileVideoStream (v) .start () 
  fileStream = True
  time.sleep(1.0)
  while True:
    # if this is a file video stream, then we need to check if
    # there any more frames left in the buffer to process
    if fileStream and not vs.more():
      break

    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale
    # channels)
    try:
      frame = vs.read()
      #print("I have read a frame successfully!")
      frame = imutils.resize(frame, width=450)
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      # detect faces in the grayscale frame
      rects = detector(gray, 0)

      # loop over the face detections
      for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(gray, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(gray, [rightEyeHull], -1, (0, 255, 0), 1)

        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
          COUNTER += 1

        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
          # if the eyes were closed for a sufficient number of
          # then increment the total number of blinks
          if COUNTER >= EYE_AR_CONSEC_FRAMES:
            TOTAL += 1

          # reset the eye frame counter
          COUNTER = 0
    except:
        break
  vs.stop()
  return TOTAL

from moviepy.editor import *

def make_video_clips(original_file, duration):
  #Note here: Duration must be in seconds
  clip = VideoFileClip(original_file)
  total=clip.duration
  i=0
  while i<=total-duration:
      start_time=i
      end_time=i+duration
      ffmpeg_extract_subclip(original_file, start_time, end_time, targetname=str(i)+"AI_108_results.mp4") #Backend People have to change file location here in targetname
      i+=duration
  return 


In [17]:
make_video_clips(r"/content/drive/MyDrive/Depression/Ruhika/27AI_108.mp4",5)


[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/Depression/Ruhika/27AI_108.mp4 -ss 0.00 -t 5.00 -vcodec copy -acodec copy 0AI_108_results.mp4
... command successful.

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/Depression/Ruhika/27AI_108.mp4 -ss 5.00 -t 5.00 -vcodec copy -acodec copy 5AI_108_results.mp4
... command successful.

[MoviePy] Running:
>>> /usr/bin/ffmpeg -y -i /content/drive/MyDrive/Depression/Ruhika/27AI_108.mp4 -ss 10.00 -t 5.00 -vcodec copy -acodec copy 10AI_108_results.mp4
... command successful.


In [103]:
p=r"/content/shape_predictor_68_face_landmarks.dat"
v=r"/content/drive/MyDrive/Ruhika/27AI_108.mp4"
x=blink_detector(p,v)

In [104]:
x


0

In [51]:
df['More_than_thresh']=df['EAR']>0.30

In [53]:
import numpy as np

In [54]:
a=df['More_than_thresh'].to_numpy()

In [56]:
df.to_csv('Sample.csv')

In [57]:
a=list(a)

In [105]:
len(a)

401

In [63]:
maximum=0
indices=[]
for i in range(len(a)):
  if a[i]==False:
    indices.append(i)

In [64]:
indices

[3,
 4,
 38,
 39,
 40,
 74,
 75,
 76,
 77,
 187,
 188,
 189,
 190,
 191,
 196,
 197,
 198,
 199,
 200,
 206,
 207,
 218,
 219,
 220,
 221,
 222,
 223,
 266,
 267,
 268,
 269,
 270,
 305,
 306,
 307,
 308,
 309,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 333,
 334,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367]

In [65]:
indices.sort()

In [87]:
len(indices)

94

In [88]:
t=indices[50:]

In [97]:
breakpoint=[]
small=[]
for i in range(len(indices)-1):
  if indices[i+1]==indices[i]+1 and len(small)==0:
    small.append(i)
  elif i==len(indices)-2 and len(small)>0:
      small.append(i)
      breakpoint.append(small)
  elif indices[i+1]==indices[i]+1 and len(small)>0:
    continue
  elif indices[i+1]!=indices[i]+1 and len(small)>0:
    small.append(i)
    breakpoint.append(small)
    small=[]

In [98]:
breakpoint

[[0, 1],
 [2, 4],
 [5, 8],
 [9, 13],
 [14, 18],
 [19, 20],
 [21, 26],
 [27, 31],
 [32, 36],
 [37, 92]]

In [99]:
indices[36]

309

In [101]:
indices[38]

312

In [106]:
401/16=FPS

25.0625